
* Работа для агенства "Digital Scouting"
* Данные по зарплатам игроков примерно 30 ведущих футбольных лиг.

# Содержание

1. [Подготовим данные и сделаем первые выводы](#view1)
2. [Зарплатный рейтинг по возврастам](#view5)
  - 2.1 [Итоговая таблица по возрастам](#age1)
3. [Зарплатный рейтинг по позициям](#view4)
  - 3.1 [Итоговая таблица по позициям](#poz1)
4. [Национальный рейтинг](#naz1)
  - 4.1 [Национальный рейтинг итоговая таблица](#nazf)
5. [Национальный рейтинг по зп](#naz2)
  - 5.1 [Национальный рейтинг по игрокам итоговая таблица](#nazf2)
6. [Рейтинг клубов по ЗП ведомости](#view2) 

 - 6.1 [Итог таблица перцентили](#final1)
 - 6.2 [Итог таблица по игрокам](#final2)
 
7. [Рейтинг лиг по зп ведомости](#view3)
  - 7.1 [Итоговая таблица по рейтингу лиг](#zp1)


8. [Вывод](#vivod1)

## 1.Подготовим данные и сделаем первые выводы<a name="view1"></a>

In [146]:
import pandas as pd
import json
from IPython.display import display
import sklearn
from sklearn.cluster import KMeans
import numpy as np
import math
import matplotlib.pyplot as plt
import os
#увеличим вывод строк
pd.options.display.max_rows = 1000

pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.0f}'.format)

In [147]:
data = pd.read_csv('nazrat.csv',sep = ",",low_memory=False)
data = data.replace("\'", '\"')

In [148]:
display(data.head(12))
display(data.shape)

,player,net_pw,net_py,position,age,country,club,league
0,Kylian Mbappé,977432,5082648000001,F,23,France,PSG,Ligue 1
1,Lionel Messi,684203,3557853600002,F,34,Argentina,PSG,Ligue 1
2,Neymar,606008,3151241800003,F,29,Brazil,PSG,Ligue 1
3,Cristiano Ronaldo,579644,3014150700004,F,36,Portugal,Manchester United,Premier League
4,Gareth Bale,319034,1658976300005,F,32,Wales,Real Madrid,La Liga
5,Eden Hazard,293230,1524794400006,F,31,Belgium,Real Madrid,La Liga
6,Sergio Ramos,293230,1524794400007,D,35,Spain,PSG,Ligue 1
7,Frenkie de Jong,273681,1423141400008,M,24,Netherlands,Barcelona,La Liga
8,Kevin De Bruyne,265245,1379275400009,F,30,Belgium,Manchester City,Premier League
9,David de Gea,248783,1293673500010,K,31,Spain,Manchester United,Premier League


(14291, 8)

In [149]:
# с помощью лямб функции поменяем тип данных с object  на  float для того что бы произвести расчеты
#data['net_py'] = data['net_py'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('float')
#data['net_pw'] = data['net_pw'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('float')


## 5. Национальный рейтинг по зп <a name="naz2"></a>

Национальность	Наивысшая зарплата	Зарплата 10 игрока	Зарплата 30 игрока

In [150]:
zp = data.groupby(['country']).agg({'net_py':'max'})
zp.sort_values(by='net_py',ascending = False,inplace=True)
display(zp.shape)
zp.head(1)

(153, 1)

,net_py
country,
France,5082648000001


In [151]:
filter_list=[10,30]

In [152]:
#data_coutry = data.copy()
#data_coutry['rank'] = data_coutry.groupby('country')['net_py'].rank(method='dense',ascending=False)
#data_coutry_filtered = data_coutry.query("rank == @filter_list")
#data_coutry_filtered.sort_values(by = 'rank', ascending = False)
#display(data_coutry_filtered.shape)
#data_coutry_filtered.head(305)


data['rank'] = data.groupby('country')['net_py'].rank(method='dense',ascending=False)
data_coutry_filtered = data.query("rank == @filter_list")
data_coutry_filtered.sort_values(by = 'rank', ascending = False)
display(data_coutry_filtered.shape)
data_coutry_filtered.head(3)

(143, 9)

,player,net_pw,net_py,position,age,country,club,league,rank
38,Kingsley Coman,176133,915893200039,F,25,France,Bayern Munich,1.Bundesliga,10
81,Cesc Fàbregas,122375,636347500082,M,34,Spain,Monaco,Ligue 1,10
103,Fernandinho,99467,517228300104,M,36,Brazil,Manchester City,Premier League,10


In [153]:
data_pivot = pd.pivot_table(data_coutry_filtered,
               index='country',
               values="net_py",
               columns="rank"
               )

display(data_pivot.shape)
data_pivot

(89, 2)

rank,10,30
country,,
Albania,47776902718,5082609680
Algeria,121983601008,16264505907
Angola,25413204477,NaN
Argentina,325289500251,152479400780
Australia,19314105303,3617011336
Austria,124016600992,45000002848
Belgium,309854700267,98603401302
Benin,3049611649,NaN
Bosnia-Herzegovina,63024802015,20330605131


## 5.1. Национальный рейтинг по игрокам итоговая табица<a name="nazf2"></a>

In [154]:
rating_naz = data_pivot.merge(zp ,left_on='country' ,right_on = 'country')
display(rating_naz.shape)
rating_naz = rating_naz.sort_values(by='net_py',ascending = False)
rating_naz = rating_naz.fillna(0)
rating_naz 

(89, 3)

,10.0,30.0,net_py
country,,,
France,915893200039,341201900239,5082648000001
Argentina,325289500251,152479400780,3557853600002
Brazil,517228300104,276096200318,3151241800003
Portugal,318173800261,139999980861,3014150700004
Wales,103686801186,25318904485,1658976300005
Spain,636347500082,363917600211,1524794400007
Belgium,309854700267,98603401302,1524794400006
Netherlands,259217000358,139856600863,1423141400008
Poland,142314100832,45743802819,1239149600011


In [170]:
en = data.query('country == "England"')
en.sort_values(by='rank',ascending = True).head(11)

,player,net_pw,net_py,position,age,country,club,league,rank
11,Jadon Sancho,232090,1206866000012,F,21,England,Manchester United,Premier League,1
25,Raheem Sterling,198934,1034456500026,F,27,England,Manchester City,Premier League,2
24,Jack Grealish,198934,1034456500025,F,26,England,Manchester City,Premier League,3
42,John Stones,165778,862047100043,D,27,England,Manchester City,Premier League,4
67,Harry Kane,132623,689637700068,F,28,England,Tottenham,Premier League,5
65,Marcus Rashford,132623,689637700066,F,24,England,Manchester United,Premier League,6
78,Harry Maguire,125899,654673500079,D,28,England,Manchester United,Premier League,7
77,Ben Chilwell,125899,654673500078,D,25,England,Chelsea,Premier League,8
84,Trent Alexander-Arnold,119407,620915100085,D,23,England,Liverpool,Premier League,9
109,Luke Shaw,99467,517228300110,D,26,England,Manchester United,Premier League,10


In [156]:
ger = data.query('country == "Germany"')
ger

,player,net_pw,net_py,position,age,country,club,league,rank
14,Manuel Neuer,217576,1131397400015,K,36,Germany,Bayern Munich,1.Bundesliga,1
16,Thomas Müller,212494,1104967700017,F,32,Germany,Bayern Munich,1.Bundesliga,2
18,Leroy Sané,207216,1077521400019,F,26,Germany,Bayern Munich,1.Bundesliga,3
21,Toni Kroos,205847,1070405700022,M,32,Germany,Real Madrid,La Liga,4
23,Joshua Kimmich,202133,1051091600024,M,26,Germany,Bayern Munich,1.Bundesliga,5
35,Leon Goretzka,186494,969769200036,M,26,Germany,Bayern Munich,1.Bundesliga,6
37,Timo Werner,180385,938003700038,F,25,Germany,Chelsea,Premier League,7
92,Marco Reus,113969,592636800093,F,32,Germany,Borussia Dortmund,1.Bundesliga,8
100,Mats Hummels,103608,538760700101,D,33,Germany,Borussia Dortmund,1.Bundesliga,9
110,Kai Havertz,99467,517228300111,F,22,Germany,Chelsea,Premier League,10


In [160]:
es = data.query('country == "Spain"')
es

,player,net_pw,net_py,position,age,country,club,league,rank
6,Sergio Ramos,293230,1524794400007,D,35,Spain,PSG,Ligue 1,1
9,David de Gea,248783,1293673500010,K,31,Spain,Manchester United,Premier League,2
19,Sergio Busquets,206434,1073455300020,M,33,Spain,Barcelona,La Liga,3
26,Jordi Alba,195486,1016529600027,D,32,Spain,Barcelona,La Liga,4
48,Koke,150134,780694700049,M,30,Spain,Atletico Madrid,La Liga,5
64,Thiago,132623,689637700065,M,30,Spain,Liverpool,Premier League,6
69,Saúl Ñíguez,131463,683609400070,M,27,Spain,Chelsea,Premier League,7
70,Ansu Fati,130976,681074800071,F,19,Spain,Barcelona,La Liga,8
80,Isco,123156,640413600081,F,29,Spain,Real Madrid,La Liga,9
81,Cesc Fàbregas,122375,636347500082,M,34,Spain,Monaco,Ligue 1,10


In [162]:
be = data.query('country == "Belgium"')
be

,player,net_pw,net_py,position,age,country,club,league,rank
5,Eden Hazard,293230,1524794400006,F,31,Belgium,Real Madrid,La Liga,1
8,Kevin De Bruyne,265245,1379275400009,F,30,Belgium,Manchester City,Premier League,2
15,Romelu Lukaku,215628,1121264100016,F,28,Belgium,Chelsea,Premier League,3
94,Thibaut Courtois,110841,576372300095,K,29,Belgium,Real Madrid,La Liga,4
135,Michy Batshuayi,90901,472686300136,F,28,Belgium,Besiktas,Super Lig,5
142,Dries Mertens,87969,457438300143,F,34,Belgium,Napoli,Serie A,6
154,Axel Witsel,82886,431008600155,M,33,Belgium,Borussia Dortmund,1.Bundesliga,7
161,Christian Benteke,79527,413541500162,F,31,Belgium,Crystal Palace,Premier League,8
205,Yannick Carrasco,70962,369000200206,F,28,Belgium,Atletico Madrid,La Liga,9
266,Leander Dendoncker,59587,309854700267,M,26,Belgium,Wolverhampton,Premier League,10


## Вывод:
* По максимально зарплате в клубе лидеры Франция, Аргентина,Бразилия и Португалия, думаю можно без проблем называть этих игроков. Франция сильно в отрыве по игрокам из всех корзин. 

* Только у Бразили и Испании оказались 200е и 100е игроки у которых больши разброс по зп.

* Нули представлены здесь как замена пропущенным значениям, не все игроки  дотягивают до отметки 100е и 200е и ранк кидает их в другую корзину. Для того что бы проверить это оставил вычесления выше. 

* Получается, что несмотря на количество игроков зп могут быть сильно в разных диапозонах.

##  Найдем суммы зарплат 
(платежные ведомости) всех команд. 
* найти где граница между топ-клубами и просто сильными командами.

In [72]:
data_new = pd.read_csv('rank.csv',sep = ",",low_memory=False)
data_new = data_new.replace("\'", '\"')

In [73]:
# с помощью лямб функции поменяем тип данных с object  на  float для того что бы произвести расчеты
#data_new['net_py'] = data_new['net_py'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('float')
#data_new['net_pw'] = data_new['net_pw'].apply(lambda x: x.replace('$', '').replace(',', '')).astype('float')



In [74]:
# в расчет берем зп за год
total_salary = data_new.groupby(['club']).agg({'net_py':'sum'})
total_salary.sort_values(by='net_py',ascending = False,inplace=True)

In [75]:
total_salary.reset_index()

,club,net_py
0,PSG,240333006
1,Manchester United,163547817
2,Real Madrid,157389278
3,Bayern Munich,128184379
4,Chelsea,115478149
5,Barcelona,114644204
6,Juventus,102262869
7,Manchester City,99485060
8,Liverpool,93089032
9,Inter Milan,85693440


## Вывод:
По таблице можно сказать что суммы от 48569786 млн евро зарплатной ведомости - это топ клубы, после этой отметки категория сильные клубы. ПСЖ вне конкуренции - сильный отрыв по зп. Остальные топ 4 имеют примерно одинаковую зп ведомость, между остальными топ клубами разница существенная.

Наш топ 5 довольно очевидный:
* PSG	240439745
* Manchester United	163547819
* Real Madrid	157389280
* Bayern Munich	128184381
* Chelsea 116225653

## Посмотрим на таблицу более подробно
Нужно найти:

* где будут российские клубы, Аякс, Спортинг, Зальцбург, Шахтер 

* Где будет первый клуб МЛС, первый мексиканский и т.д.

In [76]:
#найдем сумму платежной ведомости клуба
total_sum_test = data_new.groupby(['club','league']).agg({'net_py':'sum'})
total_sum_test.sort_values(by='net_py',ascending = False,inplace=True)
total_sum_test.head(3)

,,net_py
club,league,
PSG,Ligue 1,240333006
Manchester United,Premier League,163547817
Real Madrid,La Liga,157389278


In [77]:
#найдем максимальную зп
total_sum_2 = data_new.groupby(['club','league']).agg({'net_py':'max'})
total_sum_2.sort_values(by='net_py',ascending = False,inplace=True)

In [78]:
test_two = total_sum_test.merge(total_sum_2,left_on=['club','league'],right_on=['club','league'],how='right')
display(test_two.shape)

(488, 2)

In [79]:
total = test_two.rename({'net_py_x':'net_py','net_py_y':'net_pw'}, axis=1)

In [80]:
display(total.shape)
display(total.head(4))



(488, 2)

,,net_py,net_pw
club,league,,
PSG,Ligue 1,240333006,50826480
Manchester United,Premier League,163547817,30141507
Real Madrid,La Liga,157389278,16589763
Barcelona,La Liga,114644204,14231414


In [81]:
#сохраняем таблицу с чтобы отправить себе и потом загрузим снова с индексами
total.to_csv(r'total_ver2.csv')

In [82]:
#проставим индексы места в рейтинге
dt = pd.read_csv('total_ver2.csv',sep = ",",index_col = 0,low_memory=False)
dt = dt.replace("\'", '\"')

In [83]:
dt.groupby('club')
dt = dt.sort_values(by='net_py',ascending = False)
dt.reset_index()

,club,league,net_py,net_pw
0,PSG,Ligue 1,240333006,50826480
1,Manchester United,Premier League,163547817,30141507
2,Real Madrid,La Liga,157389278,16589763
3,Bayern Munich,1.Bundesliga,128184379,12391496
4,Chelsea,Premier League,115478149,11212641
5,Barcelona,La Liga,114644204,14231414
6,Juventus,Serie A,102262869,8132237
7,Manchester City,Premier League,99485060,13792754
8,Liverpool,Premier League,93089032,7583603
9,Inter Milan,Serie A,85693440,7115707


* первый клуб не из топ 5 это Бешикташ 26 место, следом Зенит 33 место.
* Фламенго 42 место * Спартак  - 48 место * Краснодар - 68 место * Аякс - 63 место  * Локомотив - 66 место * ЦСКА- 72 место * Динамо - 74 место * Рейнджерс - 77 место * Мексиканский UANL - 80 место * Орландо - 91 место *  Арсенал Тула - 114 место * Спортинг - 117 * Ростов - 122 * Сочи - 123 * Рубин - 135 * Ахмат - 143 * * Базель - 160 * Уфа 182 * Нижний - 206 * Урал - 217  * Крылья - 275 

## 6.Рейтинг клубов по ЗП ведомости<a name="view2"></a>

Задача: Собрать таблицу с колонками

- место в рейтинге - клуб - место в лиге по зп - размер ведомости net - максимальная зп - Перцентиль 90	- Перцентиль 80 - Перцентиль 60	- Перцентиль 40

Шаги:
1.Собрать зп ведомость по лигам и распределить клубы 
2.Собрать размезмер зп ведомости (сделано)
3.Найти максимальную зп
4.Найти перцентили и добавить в таблицу

In [84]:
percentil_40 = data_new.groupby('club')['net_py'].quantile(0.4)
percentil_60 = data_new.groupby('club')['net_py'].quantile(0.6)
percentil_80 = data_new.groupby('club')['net_py'].quantile(0.8)
percentil_90 = data_new.groupby('club')['net_py'].quantile(0.9)

In [85]:
#соберем перцентили
data_perc = pd.DataFrame()
data_perc['percentil40'] = percentil_40
data_perc['percentil60'] = percentil_60
data_perc['percentil80'] = percentil_80 
data_perc['percentil90'] = percentil_90
display(data_perc.shape)

(488, 4)

In [86]:
#максимальная зарпатная ведомость по команде
maxzp = data_new.groupby(['club','league']).agg({'net_pw':'max'})
maxzp.sort_values(by='net_pw',ascending = False,inplace=True)
display(maxzp.shape)
maxzp 

(488, 1)

,,net_pw
club,league,
PSG,Ligue 1,977432
Manchester United,Premier League,579644
Real Madrid,La Liga,319034
Barcelona,La Liga,273681
Manchester City,Premier League,265245
Bayern Munich,1.Bundesliga,238298
Chelsea,Premier League,215628
Atletico Madrid,La Liga,195486
Arsenal,Premier League,165778


In [87]:
#соеденим таблицы
test_2 = maxzp.merge(dt,left_on='club',right_on='club',how='right')
display(test_2.shape)
test_2 

(488, 4)

,net_pw_x,league,net_py,net_pw_y
club,,,,
PSG,977432,Ligue 1,240333006,50826480
Manchester United,579644,Premier League,163547817,30141507
Real Madrid,319034,La Liga,157389278,16589763
Bayern Munich,238298,1.Bundesliga,128184379,12391496
Chelsea,215628,Premier League,115478149,11212641
Barcelona,273681,La Liga,114644204,14231414
Juventus,156389,Serie A,102262869,8132237
Manchester City,265245,Premier League,99485060,13792754
Liverpool,145839,Premier League,93089032,7583603


In [88]:
#соеденим таблицы
df_merge = test_2.merge(data_perc,left_on='club' ,right_on = 'club' )
#df_merge.sort_values(by=['net_py'],ascending=False).reset_index()
df_merge.groupby('club')
display(df_merge.shape)
df_merge 

(488, 8)

,net_pw_x,league,net_py,net_pw_y,percentil40,percentil60,percentil80,percentil90
club,,,,,,,,
PSG,977432,Ligue 1,240333006,50826480,527578.60,4159639.00,8050913.20,10673560.80
Manchester United,579644,Premier League,163547817,30141507,2963513.00,4205343.60,8340758.20,11753982.20
Real Madrid,319034,La Liga,157389278,16589763,4116945.00,4808185.00,10704056.00,11344470.50
Bayern Munich,238298,1.Bundesliga,128184379,12391496,1813488.80,4525590.20,9697691.60,10857552.90
Chelsea,215628,Premier League,115478149,11212641,3421662.80,4135415.20,5172285.60,7599276.90
Barcelona,273681,La Liga,114644204,14231414,1500397.60,2976398.60,5133475.20,8579509.60
Juventus,156389,Serie A,102262869,8132237,3049589.00,4066117.60,5895872.20,6810748.00
Manchester City,265245,Premier League,99485060,13792754,3098547.20,4135415.80,5312139.40,9827336.10
Liverpool,145839,Premier League,93089032,7583603,2141253.00,3448187.80,5587029.80,6346597.00


In [89]:
df_merge = df_merge.drop(['net_pw_x'],axis=1)

In [90]:
final = df_merge
#.rename({'club':'клуб','net_pw_y':'макс зп','ligue':'лига','net_py':'размер ведомости','percentil40':'перцентиль40','percentil60':'перцентиль60','percentil80':'перцентиль80','percentil90':'перцентиль90'}, axis=1)
display(final.shape)
final.head(2)

(488, 7)

,league,net_py,net_pw_y,percentil40,percentil60,percentil80,percentil90
club,,,,,,,
PSG,Ligue 1,240333006,50826480,527578.60,4159639.00,8050913.20,10673560.80
Manchester United,Premier League,163547817,30141507,2963513.00,4205343.60,8340758.20,11753982.20


In [91]:
#создадим фильтр лист для ранк
filter_list_rank=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]

In [92]:
dt_ranked = final.copy()
dt_ranked['rank'] = final.groupby('league')['net_py'].rank(method='dense',ascending=False)
dt_rank_filtered = dt_ranked.query("rank == @filter_list_rank")
#dt_rank_filtered.sort_values(by = ['net_py'], ascending = False)
dt_rank_filtered.head(1)

,league,net_py,net_pw_y,percentil40,percentil60,percentil80,percentil90,rank
club,,,,,,,,
PSG,Ligue 1,240333006,50826480,527578.60,4159639.00,8050913.20,10673560.80,1.00


### 6.1 Итог таблица перцентили<a name="final1"></a>

In [93]:
dt_rank_filtered = dt_rank_filtered.rename({'club':'клуб','ligue':'лига','net_pw_y':'макс зп','net_py':'размер ведомости','percentil40':'перцентиль40','percentil60':'перцентиль60','percentil80':'перцентиль80','percentil90':'перцентиль90'}, axis=1)
dt_rank_filtered = dt_rank_filtered.sort_values(by = 'размер ведомости', ascending = False)
dt_rank_filtered

,league,размер ведомости,макс зп,перцентиль40,перцентиль60,перцентиль80,перцентиль90,rank
club,,,,,,,,
PSG,Ligue 1,240333006,50826480,527578.60,4159639.00,8050913.20,10673560.80,1.00
Manchester United,Premier League,163547817,30141507,2963513.00,4205343.60,8340758.20,11753982.20,1.00
Real Madrid,La Liga,157389278,16589763,4116945.00,4808185.00,10704056.00,11344470.50,1.00
Bayern Munich,1.Bundesliga,128184379,12391496,1813488.80,4525590.20,9697691.60,10857552.90,1.00
Chelsea,Premier League,115478149,11212641,3421662.80,4135415.20,5172285.60,7599276.90,2.00
Barcelona,La Liga,114644204,14231414,1500397.60,2976398.60,5133475.20,8579509.60,2.00
Juventus,Serie A,102262869,8132237,3049589.00,4066117.60,5895872.20,6810748.00,1.00
Manchester City,Premier League,99485060,13792754,3098547.20,4135415.80,5312139.40,9827336.10,3.00
Liverpool,Premier League,93089032,7583603,2141253.00,3448187.80,5587029.80,6346597.00,4.00


### Теперь соберем таблицу по игрокам

In [94]:
# отсортируем лиги по зп
total_ligue = dt.groupby(['league']).agg({'net_py':'sum'})
total_ligue.sort_values(by='net_py',ascending = False,inplace=True)
total_ligue.head(2)

,net_py
league,
Premier League,1076895771
La Liga,670741782


In [95]:
filter_list=[1,3,7,12,18]

#df_new.query("rank == @filter_list")

In [96]:
data_ranked = data_new.copy()
data_ranked['rank'] = data_new.groupby('club')['net_py'].rank(method='dense',ascending=False)
data_rank_filtered = data_ranked.query("rank == @filter_list")
display(data_rank_filtered.shape)
data_rank_filtered.sort_values(by = ['club','net_py'], ascending = False)

(3020, 9)

,player,net_pw,net_py,position,age,country,club,league,rank
1036,Ante Coric,21894,1138513,F,24,Croatia,Zurich,Super League,1.00
3129,Yanick Brecher,7624,396447,K,28,Switzerland,Zurich,Super League,3.00
3951,Nikola Boranijasevic,5865,304957,D,29,Serbia,Zurich,Super League,7.00
5396,Mirlind Kryeziu,3714,193141,D,25,Kosovo,Zurich,Super League,12.00
8106,Blaz Kramer,1759,91488,F,25,Slovenia,Zurich,Super League,18.00
...,...,...,...,...,...,...,...,...,...
52,Zlatan Ibrahimovic,136841,7115707,F,40,Sweden,AC Milan,Serie A,1.00
175,Theo Hernández,78195,4066118,D,24,France,AC Milan,Serie A,3.00
298,Mike Maignan,54736,2846283,K,26,France,AC Milan,Serie A,7.00
707,Brahim Díaz,29323,1524795,F,22,Spain,AC Milan,Serie A,12.00


In [97]:
statistic_pivot = pd.pivot_table(data_rank_filtered  ,
               index=['league','club'],
               values="net_py",
               columns="rank"
               )

display(statistic_pivot.shape)

(488, 5)

In [98]:
final_player = statistic_pivot.merge(dt_rank_filtered,left_on=['league','club'],right_on=['league','club'],how='left').reset_index()
final_player = final_player[['league','club', 'rank','размер ведомости' ,1.0, 3.0, 7.0, 12.0, 18.0]]
#final_player = final_player.sort_values(by = ['net_py'], ascending = False)
final_player = final_player.fillna(0)

## 6.2.Итог таблица по игрокам<a name="final2"></a>

In [108]:
pd.options.display.max_rows = 1000
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.0f}'.format)

final_player_2 = final_player.rename({'league':'лига','club':'клуб','rank':'ранг','net_py':'размер ведомости',1.0:'зп 1 игрока',3.0:'зп 3 игрока',7.0:'зп 7 игрока',12.0:'зп 12 игрока',18.0:'зп 18 игрока'}, axis=1)
final_player_2 = final_player.sort_values(by = 'размер ведомости', ascending = False)
final_player_2.reset_index()

,index,league,club,rank,размер ведомости,1.0,3.0,7.0,12.0,18.0
0,299,Ligue 1,PSG,1,240333006,50826480,31512418,8132237,6709095,3354548
1,364,Premier League,Manchester United,1,163547817,30141507,12068660,8270830,4310235,3098547
2,192,La Liga,Real Madrid,1,157389278,16589763,11710421,9606205,4798020,4066118
3,3,1.Bundesliga,Bayern Munich,1,128184379,12391496,11049677,9697691,4310086,1504464
4,357,Premier League,Chelsea,2,115478149,11212641,9380037,5172285,4135415,3050321
5,180,La Liga,Barcelona,2,114644204,14231414,10165296,5336780,4391408,1951737
6,427,Serie A,Juventus,1,102262869,8132237,7115707,5082649,4066117,2541324
7,363,Premier League,Manchester City,3,99485060,13792754,10344564,5172282,3797830,1591472
8,362,Premier League,Liverpool,4,93089032,7583603,6896377,5172283,3448188,2073736
9,426,Serie A,Inter Milan,2,85693440,7115707,6099178,5082647,3252895,2033058


## 7.Рейтинг лиг по зп ведомости<a name="view3"></a>
Место в рейтинге - Лига	- Максимальная зп -  Перцентиль 90 -Перцентиль 80 -Перцентиль 60 -Перцентиль 40

In [100]:
liga = data_new.groupby(['league']).agg({'net_py':'max'})
liga.sort_values(by='net_py',ascending = False,inplace=True)
display(liga.shape)

(25, 1)

In [101]:
percent_40 = data_new.groupby('league')['net_py'].quantile(0.4)
percent_60 = data_new.groupby('league')['net_py'].quantile(0.6)
percent_80 = data_new.groupby('league')['net_py'].quantile(0.8)
percent_90 = data_new.groupby('league')['net_py'].quantile(0.9)

data_perc_ligue = pd.DataFrame()
data_perc_ligue['percentil40'] = percent_40
data_perc_ligue['percentil60'] = percent_60
data_perc_ligue['percentil80'] = percent_80 
data_perc_ligue['percentil90'] = percent_90
display(data_perc_ligue.shape)

(25, 4)

In [102]:
#соеденим таблицы 
rating_ligue = liga.merge(data_perc_ligue ,left_on='league' ,right_on = 'league' )
display(rating_ligue.shape)

(25, 5)

## 7.1 Итоговая таблица по рейтингу лиг<a name="zp1"></a>

In [103]:
#Рейтинг лиг по зп ведомостям
final_ligue = rating_ligue.rename({'league':'лига','net_py':'макс зп','percentil40':'перцентиль40','percentil60':'перцентиль60','percentil80':'перцентиль80','percentil90':'перцентиль90'}, axis=1)
final_ligue = final_ligue.sort_values(by=['макс зп'],ascending=False).reset_index()
final_ligue.groupby('league')
display(final_ligue.shape)
final_ligue



(25, 6)

,league,макс зп,перцентиль40,перцентиль60,перцентиль80,перцентиль90
0,Ligue 1,50826480,121984.00,274463.00,599752.00,1209670.00
1,Premier League,30141507,882543.60,1724093.00,2760962.00,4135415.00
2,La Liga,16589763,315124.00,630248.00,1463803.00,3067886.00
3,1.Bundesliga,12391496,223637.00,453372.20,914878.40,1597984.20
4,Serie A,8132237,406611.00,813223.00,1524794.00,2541324.10
5,Premier Liga,7075046,304959.00,548926.00,853885.00,1242199.00
6,Super Lig,5082648,172810.00,345620.00,677009.00,1075488.00
7,Brasileiro,3900000,50000.00,150000.00,330001.00,582000.00
8,Liga MX,3760000,100000.00,210000.00,362000.60,500000.00
9,MLS,3600000,79200.00,165216.00,330000.00,506398.80
